In [66]:
import json
import praw
import sqlite3
import sys
import time

db_connection = sqlite3.connect("./db/data.db")

insert_into_subscribed_subreddts_table_sql_string = "INSERT INTO subscribed_subreddits VALUES ( ?, ? )"

reddit_client = None

with open('config.json') as config:
    config = json.load(config)
    reddit_client = praw.Reddit(
        client_id=config["client_id"],
        client_secret=config["client_secret"],
        user_agent=config["user_agent"]
    )


In [67]:
'''
Every 30 seconds

*Get all subscribed_subreddits from the database
::For each sub_reddit in this collection:
*Get all the new submissions to this sub_reddit and add them to the database of
subscribed submissions
'''

# time.sleep(30)
get_subscribed_subreddits_sql_string = "SELECT subreddit_title FROM subscribed_subreddits"
subscribe_to_new_submissions_sql_string = "INSERT INTO subscribed_submissions VALUES (?, ?, ?)"


def update_submissions():
    cursor = db_connection.cursor()
    cursor.execute( get_subscribed_subreddits_sql_string )
    subreddit_rows = cursor.fetchall()
    
    for subreddit_row in subreddit_rows:
        # get subreddit instance
        subreddit = reddit_client.subreddit(subreddit_name)

        
        rows_to_insert = []
        
        for submission in subreddit.new():
            row = ( submission.id, subreddit.display_name, submission.title )
            rows_to_insert.append( row )

        # push posts to subscribed_posts table ( ignoring any unique constraint errors and perhaps in the future avoiding trying to update existing posts anyway )
        cursor.executemany( subscribe_to_new_submissions_sql_string, rows_to_insert )
        db_connection.commit()
    return None

In [74]:
get_subscribed_submissions_sql_string = "SELECT id from subscribed_submissions"
record_submission_statistics_sql_string = "INSERT INTO submission_records VALUES (?, ?, ?, ?, ?) "

def record_submission_statistics():
    cursor = db_connection.cursor()
    cursor.execute( get_subscribed_submissions_sql_string )
    subreddit_rows = cursor.fetchall()

    rows_to_insert = []
    for subreddit_row in subreddit_rows:
        submission = reddit_client.submission(id=subreddit_row[0])

        current_time = int( time.time() / 60 ) * 60 # rounded back to the nearest minute
        row = ( current_time, submission.id, submission.score, submission.ups, submission.downs )

        rows_to_insert.append( row )


    cursor.executemany( record_submission_statistics_sql_string, rows_to_insert )
    db_connection.commit()
    return None

In [75]:
update_submissions()
record_submission_statistics()

In [56]:
_submission = reddit_client.submission(id="eg3fi4")

In [71]:
def get_subscribed_subreddits():
    cursor = db_connection.cursor()
    cursor.execute( get_subscribed_subreddits_sql_string )
    subreddits = [ row[0] for row in cursor.fetchall() ]
    return subreddits 

In [72]:
get_subscribed_subreddits()

['AskReddit']

In [99]:
'''
    What I want to send over to the front-end is

    a lists of subreddits

    each subreddit has a list of submissions
    each submission has a time_series_of the rows
'''

def get_subreddit_submission_time_series(): # bad function name
    # for each subreddit in the database ( get subreddits from database )
    subscribed_subreddits = { subreddit_name : {} for subreddit_name in  get_subscribed_subreddits() }

    # for each submission under subreddit ( get submission under subreddit )
    for subreddit_name in subscribed_subreddits: 

        submissions = get_subscribed_subreddit_submissions( subreddit_name )
        subscribed_subreddits[ subreddit_name ]["submissions"] = { submission[0] : { "title": submission[1] }  for submission in submissions }

        for submission_id in subscribed_subreddits[ subreddit_name ]["submissions"]:
            subscribed_subreddits[ subreddit_name ]["submissions"][ submission_id ][ "data" ] = get_submission_recorded_statistics( submission_id )

    return subscribed_subreddits



In [100]:
get_subreddit_submission_time_series()

{'AskReddit': {'submissions': {'eg4r35': {'title': 'You can now make a community on mobile, what new sub are you going to create?',
    'data': [(1577579880, 'eg4r35', 2, 2, 0),
     (1577579700, 'eg4r35', 2, 2, 0),
     (1577579520, 'eg4r35', 2, 2, 0),
     (1577579280, 'eg4r35', 2, 2, 0),
     (1577579100, 'eg4r35', 2, 2, 0),
     (1577578980, 'eg4r35', 2, 2, 0),
     (1577578800, 'eg4r35', 2, 2, 0),
     (1577578620, 'eg4r35', 2, 2, 0),
     (1577578440, 'eg4r35', 2, 2, 0),
     (1577578320, 'eg4r35', 2, 2, 0),
     (1577578140, 'eg4r35', 2, 2, 0),
     (1577578020, 'eg4r35', 2, 2, 0),
     (1577577900, 'eg4r35', 2, 2, 0),
     (1577577720, 'eg4r35', 2, 2, 0),
     (1577577600, 'eg4r35', 2, 2, 0),
     (1577577480, 'eg4r35', 2, 2, 0),
     (1577577360, 'eg4r35', 2, 2, 0),
     (1577577180, 'eg4r35', 2, 2, 0),
     (1577577060, 'eg4r35', 2, 2, 0),
     (1577576940, 'eg4r35', 2, 2, 0),
     (1577576820, 'eg4r35', 2, 2, 0),
     (1577576700, 'eg4r35', 2, 2, 0),
     (1577576640, 'eg4r3

In [94]:
def get_submission_recorded_statistics( submission_id ):
    get_submission_recorded_statistics_sql_string = "select * from submission_records where id=? ORDER BY time DESC;"
    cursor = db_connection.cursor()
    cursor.execute( get_submission_recorded_statistics_sql_string, [submission_id] )
    return cursor.fetchall()

get_submission_recorded_statistics( "eg4pdo" )

[(1577578800, 'eg4pdo', 5, 5, 0),
 (1577578620, 'eg4pdo', 6, 6, 0),
 (1577578440, 'eg4pdo', 6, 6, 0),
 (1577578320, 'eg4pdo', 6, 6, 0),
 (1577578140, 'eg4pdo', 7, 7, 0),
 (1577578020, 'eg4pdo', 7, 7, 0),
 (1577577840, 'eg4pdo', 6, 6, 0),
 (1577577720, 'eg4pdo', 6, 6, 0),
 (1577577600, 'eg4pdo', 7, 7, 0),
 (1577577480, 'eg4pdo', 6, 6, 0),
 (1577577300, 'eg4pdo', 5, 5, 0),
 (1577577180, 'eg4pdo', 6, 6, 0),
 (1577577060, 'eg4pdo', 6, 6, 0),
 (1577576940, 'eg4pdo', 7, 7, 0),
 (1577576820, 'eg4pdo', 6, 6, 0),
 (1577576700, 'eg4pdo', 5, 5, 0),
 (1577576580, 'eg4pdo', 6, 6, 0),
 (1577576520, 'eg4pdo', 6, 6, 0),
 (1577576400, 'eg4pdo', 6, 6, 0),
 (1577576280, 'eg4pdo', 5, 5, 0),
 (1577576220, 'eg4pdo', 6, 6, 0),
 (1577576100, 'eg4pdo', 6, 6, 0),
 (1577575980, 'eg4pdo', 6, 6, 0),
 (1577575920, 'eg4pdo', 6, 6, 0),
 (1577575680, 'eg4pdo', 6, 6, 0),
 (1577414520, 'eg4pdo', 5, 5, 0),
 (1577412600, 'eg4pdo', 3, 3, 0)]

In [85]:
def get_subscribed_subreddit_submissions( subreddit_name ):
    get_subscribed_submissions_sql_string = "SELECT id,submission_title from subscribed_submissions where subreddit_title=?"
    cursor = db_connection.cursor()
    cursor.execute( get_subscribed_submissions_sql_string, [subreddit_name] )
    return cursor.fetchall()

get_subscribed_subreddit_submissions( "AskReddit" )

[('eg4r35',
  'You can now make a community on mobile, what new sub are you going to create?'),
 ('eg4r34', 'When was the moment you realized you had found your people?'),
 ('eg4r1j', 'How to promote a new subreddit?'),
 ('eg4r1f',
  'Why do you think dogs are capable of love, or is it conditioning?'),
 ('eg4r06', 'Which habit Are you proudest of breaking?'),
 ('eg4qyj',
  "Now that the 2010's are coming to an end, what was the best movie from the past decade?"),
 ('eg4qxu', 'What is your story about an animals life you have saved?'),
 ('eg4qtb',
  "Reddit, why are fat people the absolute worst? They stink and they're arrogant. Why should society allow these peoplle basic freedoms?"),
 ('eg4qrx', 'How do you know it’s worth trying again with an ex?'),
 ('eg4qrp',
  '[Serious] People whose parents murdered the other parent, what happened?'),
 ('eg4qqs', 'At what point do you think murder is justified?'),
 ('eg4qp5', 'What is your favorite life-based movie quote?'),
 ('eg4qow',
  'Physic

In [89]:
reddit_client.submission(id="eg4oq3").url

'https://www.reddit.com/r/AskReddit/comments/eg4oq3/you_run_an_inconvenience_store_what_do_you_sell/'